### Housing Project step 1
#### Data Cleaning
##### Group: Karan, Lance, Gil, Rachel, Alex, Travis

### This is a simple R script to make converting your columns a bit easier
### We can save much time using this tool

* First, we're going to read in the data from all the years. 
* __Double Check your directory path__, my .csv's happen to be <br>
in the directory upstream of my working directory.

In [1]:
# Imports
library(dplyr)

# Create a list of the years for which we have data
years <- c(1991, seq(1993, 2017, 3))

# Declare a list to store our dataframes in
df_list <- list()

# Make a list of the file names we're going to read in 
for (i in 1:length(years)) {

    # Data-only data frame (with no headers):
    data <- read.csv(paste('../nycHousing', years[i], '.csv', sep = ''), 
                     skip = 2, header = FALSE)

    # Temporary data frame from which to extract the 
    # first row of headers, nrows = 2 so we don't waste time reading
    # the whole csv again.
    tmp <- read.csv(paste('../nycHousing', years[i], '.csv', sep = ''), 
                    header = TRUE, nrows = 2)

    # Use headers from tmp for nych17:
    names(data) <- names(tmp)

    # Remove the temporary data frame:
    rm(tmp)
    
    # Append the dataframe to a list of dataframes
    df_list[[i]] <- data

} # End for loop


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



### Missing siding, bricks, etc., or sloped walls
* In all besides the 2017 X_d1 and X_d2 are separate. 2017, they're together as <br>
X_d12. 
* The following loop combines X_d1 and X_d2 into a X_d12 as the 2017 survey has <br>
so we can use it in the index.

In [2]:
for (i in 1:9) {
    df <- df_list[[i]] %>%
            select(X_d1, X_d2)

    X_d12 <- vector()
    
    for (j in 1:length(df$X_d1)) {
    
        if (df[j, 1] == 1 | df[j, 2] == 1) {
            x <- 1
            X_d12[j] <- x
        } else if (df[j, 1] == 8 | df[j, 2] == 8) {
            x <- 8
            X_d12[j] <- x
        } else {
            x <- 9
            X_d12[j] <- x
        } 
    }
    
    df_list[[i]] <- cbind(X_d12, df_list[[i]])
    df_list[[i]] <- select(df_list[[i]], -c(X_d1, X_d2)) 
}

* We've got a similar problem in the completed plumbing question:
* Here's a solve.

In [3]:
for (i in 1:9) {
    df <- df_list[[i]] %>%
            select(X_25a)

    X_25new <- vector()
    
    for (j in 1:length(df$X_25a)) {
    
        if (df[j, 1] == 0) {
            X_25new[j] <- 1
        } else {
            X_25new[j] <- 2
        } 
    }
    
    df_list[[i]] <- cbind(X_25new, df_list[[i]])
    df_list[[i]] <- select(df_list[[i]], -c(X_25a)) 
}

df_list[[10]] <- df_list[[10]] %>%
                    rename(X_25new = X_25a)

### Make your codebooks, y'all.

In [4]:
# Create dataframe codebooks for appropriate columns.
cb1 <- data.frame(borough = c(1, 2, 3, 4, 5),
                    borough_name = c("Bronx", "Brooklyn", "Manhattan",
                                       "Queens", "Staten Island"))

cb2 <- data.frame(X_d12 = c(1, 9, 8), 
                   ex_walls_missing_sloping = c("Yes", "No", NA))  

# cb3 <- data.frame(X_d2 = c(1, 9, 8), 
#                    ex_walls_sloping = c("No", "No", NA)) 

cb4 <- data.frame(X_d3 = c(1, 9, 8),
                   ex_walls_cracks = c("Yes", "No", NA)) 

cb5 <- data.frame(X_d4 = c(1, 9 ,8), 
                   ex_walls_looseMaterial = c("Yes", "No", NA)) 

cb6 <- data.frame(X_e1 = c(1, 9, 8), 
                   ex_windows_missing = c("Yes", "No", NA))  

cb7 <- data.frame(X_e2 = c(1, 9, 8), 
                   ex_windows_loose = c("Yes", "No ", NA)) 

cb8 <- data.frame(X_e3 = c(1, 9, 8), 
                   ex_windows_boarded = c("Yes", "No", NA))

# Yes means loose, broken, or missing 
cb9 <- data.frame(X_f1 = c(1, 9, 8), 
                   handrail_problem = c("Yes", "No ", NA))  

cb10 <- data.frame(X_f2 = c(1, 9, 8), 
                   stairway_problem = c("Yes", "No ", NA))

cb12 <- data.frame(X_g3 = c(1, 9, 8), 
                   floors_deeply_worn = c("Yes", "No", NA)) 

cb13 <- data.frame(X_g4 = c(1, 9, 8), 
                   floors_holes = c("Yes", "No", NA)) 

cb14 <- data.frame(X_h = c(1, 2, 3, 8), 
                  building_condition = c("Dilapidated", "Sound", 
                                         "Deteriorating", NA))

cb15 <- data.frame(X_8 = 1:4, 
                  condo_coop = c("No", "Condo", "Coop", NA))

cb16 <- data.frame(X_9a = c(1,9), 
                   rent_or_own = c("Owned", "Rented"))

cb17 <- data.frame(X_23a = c(1, 2), 
                   elevator = c("Yes", "No"))

cb18 <- data.frame(X_25new = c(1, 2), 
                    complete_plumbing = c("Yes", "No"))

cb20 <- data.frame(X_25c = c(1:3,8:9), 
                    toilet_breakdowns = c("Yes", "No", 
                                         "No toilet", 
                                         NA, "No toilet"))

cb21 <- data.frame(X_26a = c(0, 1, 2), 
                    complete_kitchen = c("Yes", "Partial", "No"))

cb22 <- data.frame(X_26c = c(1, 2, 8, 9), 
                    kitchen_function = c("Yes", "No", NA, "No"))

cb23 <- data.frame(X_27 = 1:4, 
                   type_heating_fuel = c("Fuel oil", "Utility gas", 
                                         "Electricity", "Other fuel"))

cb24 <- data.frame(X_32a = c(0, 1, 8),
                    heating_breakdown = c('Yes', 'No', NA))

cb25 <- data.frame(X_32b = c(2, 3, 4, 5, 8, 9),
                    num_heat_breakdowns = c("1", "2", "3",
                                        "4+", NA, "No Breakdowns"))

cb26 <- data.frame(X_35a = c(1, 2, 8), 
                   mice_present = c("Yes", "No", NA))

cb27 <- data.frame(X_35c = c(1, 2, 3, 4, 5, 8), 
                    exterminator_service = c("Regularly", "When needed", 
                                            "Irregularly", "No", NA, NA))
cb28 <- data.frame(X_36a = c(1, 2, 8), 
                   wall_cracks_or_holes = c("Yes", "No", NA))

cb29 <- data.frame(X_36b = c(1, 2, 8), 
                   holes_in_floors = c("Yes", "No", NA))

# Need coebooks for 37a, 37b, 39

cb30 <- data.frame(X_38a = c(1, 2, 8), 
                   water_leaks = c("Yes", "No", NA))

cb31 <- data.frame(X_39 = c(1:4,8), 
                   neighborhood_rating = c("Excellent", "Good", "Fair", "Poor", NA))

### These are the functions to do the 'ole swap-a-roo

In [5]:
# The following is a function to change numerical values to appropriate
# categorical (named) values.
#
# This function replaces the old column with the new, and drops the old. 
#
# The inputs are as follows:
# orig_df - This is the unaltered dataframe from your .csv import (df object)
# codebook_df - This is dataframe that represents your 'dictionary' (df object)
# orig_name - the original name of the column (string)
# new_name - a new (meaningful) name you would like for the column
#
# Note that old_name and new_name, have to match the column names specified in your 
# codebook dataframe object

# This function doesn't append the year. It should be used for all 
# but the last column converted 
rf_func <- function(orig_df, 
                    codebook_df, 
                    orig_name,
                    new_name) {
    
    df <- full_join(x = codebook_df, y = orig_df, by = orig_name)
    df <- select(df, -c(orig_name))
    colnames(df)[colnames(df) == orig_name] <- new_name
    return(df)
}

### dplyr reordering

* I was seeing some unusual results while looking at the 'head' of our <br>
aggregated dataframe. As it turns out, this is because during the join <br>
function, dplyr reorders the rows. It made me realize that the columns were <br>
each reordering the rows individually, before the cbind, ruining the data. <br>
I'm going to refactor the code to solve.
* Below 

In [6]:
head(df_list[[2]])
test1 <- rf_func(df_list[[2]], cb2, 'X_d12', 'ex_walls_missing_sloping')
test1 <- rf_func(test1, cb4, 'X_d3', 'ex_walls_cracks')
head(test1)

df_list[[2]] %>%
    select(X_d12) %>%
    count(X_d12 == 1)

test1 %>%
    select(ex_walls_missing_sloping) %>%
    count(ex_walls_missing_sloping == 'Yes')

df_list[[2]] %>%
    select(X_d3) %>%
    count(X_d3 == 1)

test1 %>%
    select(ex_walls_cracks) %>%
    count(ex_walls_cracks == 'Yes')

df_list[[2]] %>%
    select(X_d12, X_d3) %>%
    count(X_d12 == 1 & X_d3 == 1)

test1 %>%
    select(ex_walls_missing_sloping, ex_walls_cracks) %>%
    count(ex_walls_missing_sloping == 'Yes' & ex_walls_cracks == 'Yes')

X_25new,X_d12,recordtype,borough,X_d3,X_d4,X_d5,X_d6,X_e1,X_e2,⋯,hhissi,hhincdis,hhincva,sub,hhweight,seqn,year,linkage,sba,geo_id2
1,9,1,3,9,9,1,9,9,9,⋯,7800,999999,999999,8,1572695,24,93,1,308,3603803
1,1,1,3,9,9,9,9,1,1,⋯,999999,999999,999999,7,1549944,25,93,1,307,3603802
1,9,1,3,9,9,1,9,9,9,⋯,6240,999999,999999,7,1578279,34,93,2,307,3603802
1,1,1,1,9,9,9,9,9,1,⋯,3000,999999,999999,1,1337616,38,93,1,101,3603710
1,9,1,3,9,9,1,9,9,9,⋯,4200,999999,999999,8,14117340,46,93,2,308,3603803
1,9,1,1,9,9,1,9,9,9,⋯,6200,999999,999999,1,7198430,51,93,2,101,3603710


ex_walls_cracks,ex_walls_missing_sloping,X_25new,recordtype,borough,X_d4,X_d5,X_d6,X_e1,X_e2,⋯,hhissi,hhincdis,hhincva,sub,hhweight,seqn,year,linkage,sba,geo_id2
Yes,Yes,1,1,3,1,9,9,1,1,⋯,5460,999999,999999,8,17156324,246,93,1,308,3603803
Yes,Yes,1,1,2,9,9,9,1,1,⋯,3600,999999,999999,4,1437938,1170,93,2,204,3604002
Yes,Yes,1,1,3,1,9,9,1,1,⋯,10224,999999,999999,8,15722196,1418,93,1,308,3603803
Yes,Yes,1,1,3,9,9,9,9,9,⋯,999999,999999,999999,8,15658576,2530,93,1,308,3603803
Yes,Yes,1,1,3,1,9,9,1,1,⋯,999999,999999,999999,6,19855012,2671,93,1,306,3603805
Yes,Yes,1,1,2,9,9,9,1,1,⋯,5124,999999,999999,4,1503955,3543,93,2,204,3604002


X_d12 == 1,n
FALSE,15552
TRUE,248


"ex_walls_missing_sloping == ""Yes""",n
FALSE,15495
TRUE,248
NA,57


X_d3 == 1,n
FALSE,15698
TRUE,102


"ex_walls_cracks == ""Yes""",n
FALSE,15641
TRUE,102
NA,57


X_d12 == 1 & X_d3 == 1,n
FALSE,15759
TRUE,41


"ex_walls_missing_sloping == ""Yes"" & ex_walls_cracks == ""Yes""",n
FALSE,15702
TRUE,41
NA,57


### Use the above functions to clean 'em up. Then slap 'em together.
* __Notice that you'll have to change the parameters__ of the function<br>
calls to the terms referenced in your codebooks.
* This will be great to use for visualizations, but not so good for building<br>
an index. You cannot execute mathematical operations on the data in this <br>
form, rendering it pretty useless in terms of calculating a quality index.<br>
* We'll end up converting it back to integers (or maybe modifying it differently<br>
with NA's for some of the categories to create a weighted numerical index?
* You tell me.

In [12]:
# Declare a list to capture the cleaned dataframes
clean_df_list <- list()

# Counter
i <- 1

# Iterate through the uncleaned dataframes, clean and factor them
for (dataframe in df_list) {
    
    # Use the appropriate function with parameters to rename your answers
    # and drop the unnecessary columns
    df <- rf_func(dataframe, cb1, 'borough', 'borough_name')
    df <- rf_func(df, cb2, 'X_d12', 'ex_walls_missing_sloping')
    df <- rf_func(df, cb4, 'X_d3', 'ex_walls_cracks')
    df <- rf_func(df, cb5, 'X_d4', 'ex_walls_looseMaterial')
    df <- rf_func(df, cb6, 'X_e1', 'ex_windows_missing')  
    df <- rf_func(df, cb7, 'X_e2', 'ex_windows_loose')
    df <- rf_func(df, cb8, 'X_e3', 'ex_windows_boarded')
    df <- rf_func(df, cb9, 'X_f1', 'handrail_problem')
    df <- rf_func(df, cb10, 'X_f2', 'stairway_problem')
    df <- rf_func(df, cb12, 'X_g3', 'floors_deeply_worn')
    df <- rf_func(df, cb13, 'X_g4', 'floors_holes')
    df <- rf_func(df, cb14, 'X_h', 'building_condition')
    df <- rf_func(df, cb15, 'X_8', 'condo_coop')
    df <- rf_func(df, cb16, 'X_9a', 'rent_or_own')
    df <- rf_func(df, cb17, 'X_23a', 'elevator')    
    df <- rf_func(df, cb18, 'X_25new', 'complete_plumbing')
    
# # Why, oh Why, do these three columns add ghost rows during conversion?!
# # Figured it out, the 2017 codebook has fewer factors as options.
# # It synthesizes a new row for each unaccounted for factor. weird.
# # Fix coming soon. 

#     df <- rf_func(df, cb20, 'X_25c', 'toilet_breakdowns')
#     df <- rf_func(df, cb21, 'X_26a', 'complete_kitchen') 

    
    
    df <- rf_func(df, cb22, 'X_26c', 'kitchen_function')
    df <- rf_func(df, cb23, 'X_27', 'type_heating_fuel')
    df <- rf_func(df, cb24, 'X_32a', 'heating_breakdown')
    df <- rf_func(df, cb25, 'X_32b', 'num_heat_breakdowns')
    df <- rf_func(df, cb26, 'X_35a', 'mice_present')
    df27 <- rf_func(df, cb27, 'X_35c', 'exterminator_service')
    df28 <- rf_func(df, cb28, 'X_36a', 'wall_cracks_or_holes')
    df29 <- rf_func(df, cb29, 'X_36b', 'holes_in_floors')
    df30 <- rf_func(df, cb30, 'X_38a', 'water_leaks')
    df31 <- rf_func(df, cb31, 'X_39', 'neighborhood_rating')
            
    # Append the clean dataframe to the list
    clean_df_list[[i]] <- df
    
    # Increase the counter
    i <- i + 1
}

# Row bind the list of cleaned dataframes, in order
aggregated_df <- bind_rows(clean_df_list)

# Factor the year column
aggregated_df$year <- as.factor(aggregated_df$year)

# aggregated_df <- aggregated_df %>%
#                     select(year, borough_name, ex_walls_missing_sloping, ex_walls_cracks,
#                           ex_walls_looseMaterial, ex_windows_missing, ex_windows_loose,
#                           ex_windows_boarded, handrail_problem, stairway_problem,
#                           floors_deeply_worn, floors_holes, building_condition, 
#                           condo_coop, rent_or_own, elevator, complete_plumbing,
#                           kitchen_function, type_heating_fuel, heating_breakdown,
#                           num_heat_breakdowns, mice_present)

# Show some proof that this works
print(c('total samples: ', length(aggregated_df$complete_plumbing)))
# print(c('Sample comparison: ', ))
# head(aggregated_df)
# str(aggregated_df)

[1] "total samples: " "156230"         


In [8]:
sum <- 0

for (i in 1:10) {
    sum <- sum + length(df_list[[i]]$borough)
}    

sum

[1] 156230